# Implementing a deep CNN using TensorFlow
## CNN on MNIST

In [5]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [6]:
# loading data
mnist_bldr = tfds.builder('mnist')
mnist_bldr.download_and_prepare()
datasets = mnist_bldr.as_dataset(shuffle_files=False)
mnist_train_orig = datasets['train']
mnist_test_orig = datasets['test']

In [7]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20

mnist_train = mnist_train_orig.map(
    lambda item: (tf.cast(item['image'], tf.float32)/255.0, 
                  tf.cast(item['label'], tf.int32)))

mnist_test = mnist_test_orig.map(
    lambda item: (tf.cast(item['image'], tf.float32)/255.0, 
                  tf.cast(item['label'], tf.int32)))

tf.random.set_seed(1)

mnist_train = mnist_train.shuffle(buffer_size=BUFFER_SIZE,
                                  reshuffle_each_iteration=False)

mnist_valid = mnist_train.take(10000).batch(BATCH_SIZE)
mnist_train = mnist_train.skip(10000).batch(BATCH_SIZE)

## Constructing a CNN in Keras

In [8]:
model = tf.keras.Sequential()

# conv layer
model.add(tf.keras.layers.Conv2D(
            filters=32, kernel_size=(5, 5),
            strides=(1, 1), padding='same',
            data_format='channels_last',
            name='conv_1', activation='relu'))

# maxpooling layer
model.add(tf.keras.layers.MaxPool2D(
            pool_size=(2, 2), name='pool_1'))

# conv layer
model.add(tf.keras.layers.Conv2D(
            filters=64, kernel_size=(5, 5),
            strides=(1, 1), padding='same',
            data_format='channels_last',
            name='conv_2', activation='relu'))

# maxpooling layer
model.add(tf.keras.layers.MaxPool2D(
            pool_size=(2, 2), name='pool_2'))

model.compute_output_shape(input_shape=(16, 28, 28, 1))

TensorShape([16, 7, 7, 64])

In [9]:
# flattening - before fully connected layer will be used
model.add(tf.keras.layers.Flatten())

model.compute_output_shape(input_shape=(16, 28, 28, 1))

TensorShape([16, 3136])

In [11]:
model.add(tf.keras.layers.Dense(units=1024, name='fc_1', activation='relu'))

model.add(tf.keras.layers.Dropout(rate=0.5))

model.add(tf.keras.layers.Dense(units=10, name='fc_2', activation='softmax'))

In [12]:
tf.random.set_seed(1)

model.build(input_shape=(None, 28, 28, 1))
model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss = tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy'])

In [13]:
history = model.fit(mnist_train, epochs=NUM_EPOCHS, validation_data=mnist_valid, shuffle=True)

Epoch 1/20
782/782 [==============================] - 200s 256ms/step - loss: 0.1444 - accuracy: 0.9554 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
782/782 [==============================] - 192s 246ms/step - loss: 0.0490 - accuracy: 0.9855 - val_loss: 0.0346 - val_accuracy: 0.9890
Epoch 3/20
782/782 [==============================] - 205s 262ms/step - loss: 0.0319 - accuracy: 0.9897 - val_loss: 0.0407 - val_accuracy: 0.9874
Epoch 4/20
782/782 [==============================] - 266s 340ms/step - loss: 0.0247 - accuracy: 0.9923


KeyboardInterrupt: 

In [ ]:
hist = history.history
x_arr = np.arange(len(hist['loss'])) + 1

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, hist['loss'], '-o', label='Train loss')
ax.plot(x_arr, hist['val_loss'], '--<', label='Validation loss')
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Loss', size=15)
ax.legend(fontsize=15)
ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, hist['accuracy'], '-o', label='Train acc.')
ax.plot(x_arr, hist['val_accuracy'], '--<', label='Validation acc.')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Accuracy', size=15)

plt.savefig('mnist_cnn.png', dpi=300)
plt.show()

In [ ]:
test_results = model.evaluate(mnist_test.batch(20))
print('Test Acc.: {:.2f}\%'.format(test_results[1]*100))

In [ ]:
batch_test = next(iter(mnist_test.batch(12)))

preds = model(batch_test[0])

tf.print(preds.shape)
preds = tf.argmax(preds, axis=1)

print(preds)

In [ ]:
fig = plt.figure(figsize=(12, 4))

for i in range(12):
    ax = fig.add_subplot(2, 6, i+1)
    ax.set_xticks([]); ax.set_yticks([])
    img = batch_test[0][i, :, :, 0]
    ax.imshow(img, cmap='gray_r')
    ax.text(0.9, 0.1, '{}'.format(preds[i]), 
            size=15, color='blue',
            horizontalalignment='center',
            verticalalignment='center', 
            transform=ax.transAxes)
    
plt.savefig('mnist_cnn_result.png', dpi=300)
plt.show()

In [ ]:
if not os.path.exists('models'):
    os.mkdir('models')


model.save('models/mnist-cnn.h5')